# Geohash binning on large dataset

Uses a Python user-defined function in DuckDB to calculate spatial bins by global, hemisphere, and 1-character geohashes.

In [1]:
import duckdb
from duckdb.typing import *
import geohash_bin

In [ ]:
selected_file = "../../dev/test_lpcloud_data/single_file/merged_sorted_ddb_75mil.parquet"

## Prepare 75m granule data

There are duplicate GranuleURs in this dataset due to how complex geometries were handled by the harvesting script. These duplicates would create issues later when joining bins, so we'll create a new version with duplicates removed.  

First, create a local DB to store the calculation result so we don't lose it if we crash later.  
Then, register the geohash binning function to the database connection.

In [ ]:
db_con = duckdb.connect('geohash.db')
db_con.create_function('hashbin_wkb', geohash_bin.wkb_to_hash_path, [BLOB, BIGINT], VARCHAR)
%load_ext sql
%sql db_con --alias duckdb

/var/folders/dk/gh88d4px3f93zz2jf81_cl0m0000gp/T/ipykernel_58509/3221329991.py:2: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  db_con.create_function('hashbin_wkb', geohash_bin.wkb_to_hash_path, [BLOB, BIGINT], VARCHAR)


For convenience, create a view that references the input 75m granule parquet file.

In [10]:
%%sql
LOAD spatial;
CREATE OR REPLACE VIEW pq75m
AS FROM (
  SELECT * FROM read_parquet('{{selected_file}}')
)

Running query in 'duckdb'

Count


Granule count in the harvested data

In [26]:
%sql SELECT COUNT(*) FROM pq75m

Running query in 'duckdb'

count_star()
75000314


Complex shapes (e.g. multipolygons, or polygons with holes) were split into multiple records, one for each constituent shape. We'll subset to a single version of each GranuleUR, keeping the largest shape by area for each granule.

In [11]:
%%sql
CREATE OR REPLACE TABLE unduped AS FROM (
  SELECT DISTINCT ON(GranuleUR) *
  FROM pq75m
  ORDER BY ST_Area(geometry)
)

Running query in 'duckdb'

Count
17648544


## Apply geohash binning to deduplicated granules

The unduplicated table stores the geometry column as DuckDB Spatial's `GEOMETRY` type. That isn't supported by Python user-defined functions (UDF), so we'll convert it back to WKB to send to the binning function.

In [12]:
%%sql
CREATE OR REPLACE TABLE binned
AS FROM (
  SELECT GranuleUR, hashbin_wkb(ST_AsWKB(geometry), 1) AS hashbin FROM unduped
)

Running query in 'duckdb'

Count
17648544


Preview the binning results

In [30]:
%sql SELECT * FROM binned LIMIT 5

Running query in 'duckdb'

GranuleUR,hashbin
92916a38-b8dd-42ba-b4bf-05587a6f1236,9
0382980e-726e-433a-8031-ac491a8d2850,NE-SE
aa9a84c5-e22b-4a82-b240-e5ac3bbbf943,All
sc:g3bt.052:64110734,2
6ae0b974-13d3-4d98-8a90-248665dff540,All


## Summary of bin counts

In [55]:
import pandas as pd
bins_df = db_con.sql("SELECT * FROM binned").df()

In [56]:
i1 = sorted(bins_df['hashbin'].unique())
i2 = ['All', 'NE-NW', 'NE', 'NW', 'NE-SE', 'SW-NW', 'SW', 'SE', 'SW-SE']
i3 = i2 + [x for x in i1 if x not in i2]
cat_type = pd.CategoricalDtype(categories=i3, ordered=True)

In [57]:
bin_counts = bins_df['hashbin'].astype(cat_type).value_counts().sort_index()
bin_counts

hashbin
All      9320146
NE-NW     724015
NE        672565
NW        734480
NE-SE     850102
SW-NW     702775
SW        418319
SE        374589
SW-SE     581949
0          62657
1          56889
2         118231
3          56586
4          66850
5          66470
6         129823
7          79986
8          79953
9         289086
b         141301
c          89618
d         252633
e         101341
f          76693
g          82358
h          35535
j          43738
k         119659
m          78774
n          38977
p          73045
q         108962
r         132801
s         143829
t         166878
u          78043
v          74501
w         195655
x         120315
y          42042
z          66375
Name: count, dtype: int64

## Join the geohash binning table and export each bin sequentially

In [23]:
outdir = 'geohash_bins'

import os
os.makedirs(os.path.join(outdir))

In [33]:
for export_bin in i3:
  db_con.sql(f"""
          COPY (
            SELECT unduped.*, binned.hashbin
            FROM unduped
            JOIN binned
            ON unduped.GranuleUR = binned.GranuleUR
            WHERE hashbin = '{export_bin}'
          )
          TO '{os.path.join(outdir, export_bin + ".parquet")}'
          (FORMAT 'parquet', ROW_GROUP_SIZE 100_000)
          """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### Summarize the output parquet files

Note the row group size parameter set to 100,000 above seems to be ignored when it would result in fewer than 10 row groups per file.

In [ ]:
%%bash
cd geohash_bins/
echo -e "$(ls [A-Z]*.parquet)" "\n" "$(ls [0-9]*.parquet)" "\n" "$(ls [a-z]*.parquet)" > filenames.txt
echo "partition,rows,row_groups,size_MB" > parquet_stats.csv
while read f; do
  echo -n "$(basename $f .parquet)"","; ~/apps/parquet-tools row-count $f| \
  tr '\n' ','; ~/apps/parquet-tools meta $f | jq '.NumRowGroups'| \
  tr '\n' ','; du -m $f | awk '{print $1}'; \
  done < filenames.txt >> parquet_stats.csv
rm filenames.txt
column -s',' -t parquet_stats.csv

partition  rows     row_groups  size_MB
All        9320146  99          1175
NE-NW      724015   10          108
NE-SE      850102   15          130
NE         672565   11          92
NW         734480   12          127
SE         374589   10          53
SW-NW      702775   11          110
SW-SE      581949   10          88
SW         418319   10          63
0          62657    10          10
1          56889    10          8
2          118231   10          15
3          56586    10          7
4          66850    10          10
5          66470    10          9
6          129823   10          15
7          79986    10          9
8          79953    10          10
9          289086   10          87
b          141301   10          21
c          89618    10          13
d          252633   10          39
e          101341   10          13
f          76693    10          17
g          82358    10          16
h          35535    10          5
j          43738    10          6
k          1196